# Name: Chain-of-Thought Reasoning with DFS

## Description: Chain-of-Thought Reasoning with DFS

## Tags: reasoning, chain of thought, depth-first search, dfs

###🧩 generated with ❤️ by Waldiez.

### Requirements

In [ ]:
import sys  # pyright: ignore

# # !{sys.executable} -m pip install -q ag2==0.9.8.post1 ag2[gemini]==0.9.8.post1

### Imports

In [ ]:
# pyright: reportUnusedImport=false,reportMissingTypeStubs=false
import csv
import importlib
import json
import os
import sqlite3
import sys
from dataclasses import asdict
from pprint import pprint
from types import ModuleType
from typing import (
    Annotated,
    Any,
    Callable,
    Coroutine,
    Dict,
    List,
    Optional,
    Set,
    Tuple,
    Union,
)

import autogen  # type: ignore
from autogen import (
    Agent,
    Cache,
    ChatResult,
    ConversableAgent,
    GroupChat,
    UserProxyAgent,
    runtime_logging,
)
from autogen.agents.experimental import ReasoningAgent
from autogen.events import BaseEvent
from autogen.io.run_response import AsyncRunResponseProtocol, RunResponseProtocol
import numpy as np
from dotenv import load_dotenv

# Common environment variable setup for Waldiez flows
load_dotenv(override=True)
os.environ["AUTOGEN_USE_DOCKER"] = "0"
os.environ["ANONYMIZED_TELEMETRY"] = "False"
#
# let's try to avoid:
# module 'numpy' has no attribute '_no_nep50_warning'"
# ref: https://github.com/numpy/numpy/blob/v2.2.2/doc/source/release/2.2.0-notes.rst#nep-50-promotion-state-option-removed
os.environ["NEP50_DEPRECATION_WARNING"] = "0"
os.environ["NEP50_DISABLE_WARNING"] = "1"
os.environ["NPY_PROMOTION_STATE"] = "weak"
if not hasattr(np, "_no_pep50_warning"):

    import contextlib
    from typing import Generator

    @contextlib.contextmanager
    def _np_no_nep50_warning() -> Generator[None, None, None]:
        """Dummy function to avoid the warning.

        Yields
        ------
        None
            Nothing.
        """
        yield

    setattr(np, "_no_pep50_warning", _np_no_nep50_warning)  # noqa

### Start logging.

In [ ]:
def start_logging() -> None:
    """Start logging."""
    runtime_logging.start(
        logger_type="sqlite",
        config={"dbname": "flow.db"},
    )


start_logging()

### Load model API keys

In [ ]:
# NOTE:
# This section assumes that a file named:
# "chain_of_thought_rea_api_keys.py"
# exists in the same directory as this file.
# This file contains the API keys for the models used in this flow.
# It should be .gitignored and not shared publicly.
# If this file is not present, you can either create it manually
# or change the way API keys are loaded in the flow.


def load_api_key_module(flow_name: str) -> ModuleType:
    """Load the api key module.

    Parameters
    ----------
    flow_name : str
        The flow name.

    Returns
    -------
    ModuleType
        The api keys loading module.
    """
    module_name = f"{flow_name}_api_keys"
    if module_name in sys.modules:
        return importlib.reload(sys.modules[module_name])
    return importlib.import_module(module_name)


__MODELS_MODULE__ = load_api_key_module("chain_of_thought_rea")


def get_chain_of_thought_rea_model_api_key(model_name: str) -> str:
    """Get the model api key.
    Parameters
    ----------
    model_name : str
        The model name.

    Returns
    -------
    str
        The model api key.
    """
    return __MODELS_MODULE__.get_chain_of_thought_rea_model_api_key(model_name)

### Models

In [ ]:
gemini_2_0_flash_llm_config: dict[str, Any] = {
    "model": "gemini-2.0-flash",
    "api_type": "google",
    "api_key": get_chain_of_thought_rea_model_api_key("gemini_2_0_flash"),
}

### Agents

In [ ]:
# pyright: reportUnnecessaryIsInstance=false

reasoning = ReasoningAgent(
    name="reasoning",
    description="A new Reasoning agent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=None,
    default_auto_reply="",
    code_execution_config=False,
    is_termination_msg=None,  # pyright: ignore
    reason_config={
        "method": "beam_search",
        "max_depth": 3,
        "forest_size": 1,
        "rating_scale": 10,
        "beam_size": 3,
        "answer_approach": "pool",
    },
    verbose=True,
    llm_config=autogen.LLMConfig(
        config_list=[
            gemini_2_0_flash_llm_config,
        ],
        cache_seed=None,
    ),
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    description="A new User proxy agent",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=None,
    default_auto_reply="",
    code_execution_config=False,
    is_termination_msg=None,  # pyright: ignore
    llm_config=False,  # pyright: ignore
)


def get_sqlite_out(dbname: str, table: str, csv_file: str) -> None:
    """Convert a sqlite table to csv and json files.

    Parameters
    ----------
    dbname : str
        The sqlite database name.
    table : str
        The table name.
    csv_file : str
        The csv file name.
    """
    conn = sqlite3.connect(dbname)
    query = f"SELECT * FROM {table}"  # nosec
    try:
        cursor = conn.execute(query)
    except sqlite3.OperationalError:
        conn.close()
        return
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    data = [dict(zip(column_names, row, strict=True)) for row in rows]
    conn.close()
    with open(csv_file, "w", newline="", encoding="utf-8") as file:
        csv_writer = csv.DictWriter(file, fieldnames=column_names)
        csv_writer.writeheader()
        csv_writer.writerows(data)
    json_file = csv_file.replace(".csv", ".json")
    with open(json_file, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)


def stop_logging() -> None:
    """Stop logging."""
    runtime_logging.stop()
    if not os.path.exists("logs"):
        os.makedirs("logs")
    for table in [
        "chat_completions",
        "agents",
        "oai_wrappers",
        "oai_clients",
        "version",
        "events",
        "function_calls",
    ]:
        dest = os.path.join("logs", f"{table}.csv")
        get_sqlite_out("flow.db", table, dest)

### Start chatting

In [ ]:
def main(
    on_event: Optional[Callable[[BaseEvent], bool]] = None,
) -> list[dict[str, Any]]:
    """Start chatting.

    Returns
    -------
    list[dict[str, Any]]
        The result of the chat session.

    Raises
    ------
    SystemExit
        If the user interrupts the chat session.
    """
    results: list[RunResponseProtocol] | RunResponseProtocol = []
    result_dicts: list[dict[str, Any]] = []
    results = user_proxy.run(
        reasoning,
        summary_method="last_msg",
        clear_history=True,
        message="What is the expected maximum dice value if you can roll a 6-sided dice three times?",
    )
    if on_event:
        if not isinstance(results, list):
            results = [results]  # pylint: disable=redefined-variable-type
        for index, result in enumerate(results):
            for event in result.events:
                try:
                    should_continue = on_event(event)
                except BaseException as e:
                    print(f"Error in event handler: {e}")
                    raise SystemExit("Error in event handler: " + str(e)) from e
                if event.type == "run_completion":
                    break
                if not should_continue:
                    raise SystemExit("Event handler stopped processing")
            result_dict = {
                "index": index,
                "messages": result.messages,
                "summary": result.summary,
                "cost": (
                    result.cost.model_dump(mode="json", fallback=str)
                    if result.cost
                    else None
                ),
                "context_variables": (
                    result.context_variables.model_dump(mode="json", fallback=str)
                    if result.context_variables
                    else None
                ),
                "last_speaker": result.last_speaker,
                "uuid": str(result.uuid),
            }
            result_dicts.append(result_dict)
    else:
        if not isinstance(results, list):
            results = [results]  # pylint: disable=redefined-variable-type
        for index, result in enumerate(results):
            result.process()
            result_dict = {
                "index": index,
                "messages": result.messages,
                "summary": result.summary,
                "cost": (
                    result.cost.model_dump(mode="json", fallback=str)
                    if result.cost
                    else None
                ),
                "context_variables": (
                    result.context_variables.model_dump(mode="json", fallback=str)
                    if result.context_variables
                    else None
                ),
                "last_speaker": result.last_speaker,
                "uuid": str(result.uuid),
            }
            result_dicts.append(result_dict)

    stop_logging()

    # pylint: disable=broad-exception-caught,too-many-try-statements
    try:
        reasoning.visualize_tree()
        if os.path.exists("tree_of_thoughts.png"):
            new_name = "reasoning_tree_of_thoughts.png"
            os.rename("tree_of_thoughts.png", new_name)
    except BaseException:
        pass
    # save the tree to json
    # pylint: disable=protected-access
    try:
        data = reasoning._root.to_dict()  # pyright: ignore
        with open("reasoning_reasoning_tree.json", "w", encoding="utf-8") as f:
            json.dump(data, f)
    except BaseException:
        pass

    return result_dicts

In [ ]:
main()